In [2]:
import numpy as np
import pandas as pd
from random import sample
import itertools
import os
import subprocess

In [42]:
global Community
global train_n
global neq
global nex
global nall
train_n=40
Community=np.ndarray.tolist(np.zeros(train_n+1))


#The time series are read in from their corresponding .txt files.
exfiles=os.listdir('./EXPLOSIONS')
eqfiles=os.listdir('./EARTHQUAKES')
nex=len(exfiles)
neq=len(eqfiles)
def dist2(e1,e2):
    minn=500
    maxx=750
    div=maxx-minn
    diststemp=[]
    for delay in range(500):
        cross=0
        
        
        for i in range(minn,maxx):
            if (e1[i]-e2[i+delay])*(e1[i+1]-e2[i+1+delay])<=0:
                cross=cross+1
        diststemp.append(cross)
    return (1-np.max(diststemp)/div)/(np.max(diststemp)/div)
    

EQ=[]
for i in eqfiles:
    eqtemp=[]
    with open('./EARTHQUAKES/'+i) as f:
        for line in f.readlines():
            if line=="\n":
                pass
            else:
                eqtemp.append(float(line))
    EQ.append(eqtemp)
EX=[]
for i in exfiles:
    extemp=[]
    with open('./EXPLOSIONS/'+i) as f:
        for line in f.readlines():
            if line=="\n":
                pass
            else:
                extemp.append(float(line))
    EX.append(extemp)


#The earthquakes and explosions are normalized.
EQnorm=[]
for e in EQ:
    mean=np.mean(e)
    std=np.var(e)**.5
    etemp=[]
    for i in range(len(e)):
        etemp.append((e[i]-mean)/std)
    
    EQnorm.append(etemp)


EXnorm=[]
for e in EX:
    mean=np.mean(e)
    std=np.var(e)**.5
    etemp=[]
    for i in range(len(e)):
        etemp.append((e[i]-mean)/std)
    
    EXnorm.append(etemp)


#The data is balanced and distance matrix is calculated.
EX=sample(EX,neq)
EXnorm=sample(EXnorm,neq)
EQXnorm=EQnorm+EXnorm
EQX=EQ+EX

nall=2*neq
Dskew=np.zeros((nall,nall))

for i in range(nall):
    for j in range(i+1,nall):
        Dskew[i][j]=(dist2(EQX[i],EQX[j]))


Dall=np.transpose(Dskew)+Dskew

#PaLD was released as an R package after I wrote this notebook. Here, I'm saving the distance matrix and reading it into an R script to get the R pald output.
print(*[np.round(x[i],2) for i in range(nall) for x in [x for x in Dall]],file=open("./dall.csv",'w'))
subprocess.call("Rscript.exe C:\Documents\pald.R", shell=True)



0

In [77]:
#This cell was used to get all the .txt files into a standard form.

""" exfiles=os.listdir('./Nevada1/EXPLOSIONS')
eqfiles=os.listdir('./Nevada1/EARTHQUAKES')


for i in exfiles:
    newfile=""
    with open('./Nevada1/EXPLOSIONS/'+i) as f:
        for line in f.readlines():
            newfile=newfile+line[16:]

    print(newfile,file=open("./Nevada1/EXPLOSIONS/exreform"+str(i),'w')) 

for i in eqfiles:
    newfile=""
    with open('./Nevada1/EARTHQUAKES/'+i) as f:
        for line in f.readlines():
            newfile=newfile+line[16:]

    print(newfile,file=open("./Nevada1/EARTHQUAKES/eqreform"+str(i),'w')) 
 """


In [36]:
def confusionfromD(Dall,reps=100):
    global Community
    Confuse=[[0,0],[0,0]]
    for dum in range(reps):
        
        #Monte Carlo cross-validation
        ind=sample(range(neq),k=int(train_n/2))+sample(range(neq,nall),int(train_n/2))

        test=sample([x for x in range(nall) if not (x in ind )],1)[0]
        D=[[Dall[i][j] for j in ind+[test]] for i in ind+[test]]


        #PaLD algorithm
        U=np.zeros((train_n+1,train_n+1))
        U=np.ndarray.tolist(U)
        for x in range(train_n+1):
            for y in range(train_n+1):
                U[x][y]=[ z for z in range(train_n+1) if D[z][x]<=D[y][x] or D[z][y]<=D[x][y]]

        C=np.ndarray.tolist(np.zeros((train_n+1,train_n+1)))
        for x in range(train_n+1):
            for w in range(train_n+1):
                C[x][w]=np.sum([ float(int(D[z][x]<D[z][y] and z==w))/(train_n)/len(U[x][y])   for [y,z] in itertools.product(range(train_n+1),range(train_n+1)) if (y!=x and z in U[x][y])   ])
	    
        
        stronk=np.mean([C[i][i] for i in range(train_n+1)])/2

        Community=np.ndarray.tolist(np.zeros(train_n+1))
        for x in range(train_n+1):
            Community[x]=[z for z in range(train_n+1) if min(C[x][z],C[z][x])>stronk]

        partitions=[]
        for i in range(train_n+1):
            pot=partition([i])
            if not pot in partitions:
                partitions.append(pot)
        maxlen=np.max([len(pot) for pot in partitions])
        testpart=partition([train_n])

        #Making the confusion matrix -- If the event is in the largest partition, I classify it as an earthquake. Generally, the explosions are more spread out and separated.
        if len(testpart)==maxlen:
            if test>=neq:
                Confuse[0][1]=Confuse[0][1]+1
                #print(test+1)
            else: 
                Confuse[0][0]=Confuse[0][0]+1
                
        else: 
            if test<neq:
                Confuse[1][0]=Confuse[1][0]+1
                #print(-test-1)
            else:
                Confuse[1][1]=Confuse[1][1]+1
    print(Confuse)



#Recursive function to find the partition of each event.
def partition(li):
			li1=set(li)
			li=set(li)
			for x in li:
				for i in range(train_n+1):
					if x in Community[i]:
						li=li.union(set(Community[i]))
			if li1==li:    
				return li
			else:
				return partition(li)



In [41]:
train_n=30
confusionfromD(Dall,reps=1000)

[[481, 79], [26, 414]]


In [43]:
train_n=30
confusionfromD(Dall,reps=1000)
#Not normalized

[[513, 75], [3, 409]]
